# Optimized Code

In [1]:
import numpy as np
import sympy as smp
from scipy import integrate
import matplotlib.pyplot as plt
from torchquad import Simpson, set_up_backend
from scipy.integrate import quad, nquad
import inspect
import pandas as pd

# For interpolation
from scipy.interpolate import RectBivariateSpline, interp2d

import warnings

# Ignorar solo DeprecationWarnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", message=".*divmax.*")

import logging
# Configuración básica del registro
logging.basicConfig(level=logging.INFO)

In [2]:
# First model
Omega_b0= 0.05
omega_b0= 0.022445
Omega_m0 = 0.32
omega_m0 = 0.143648
h = 0.67
n_s = 0.96
sigma_8 = 0.816
Omega_DE0 = 0.68
w_0 = -1
w_a = 0
gamma = 0.55
Omega_K0 = 1 - (Omega_DE0 + Omega_m0) # FLAT universe!

#[params_cosmo]
#sigma8 = 0.815584
#h = 0.670
#omega_cdm      = 0.2685628338348412
#omega_lambda   = 0.68
#pivot_scalar              = 0.05
#pivot_tensor              = 0.05
#scalar_amp(1)             = 2.12605e-09
#scalar_spectral_index(1)  = 0.96
#Omega_b = 0.05
#Omega_m = 0.32
#Omega_nu = 0.00143717
#n_s = 0.96
#A_s = 2.12605e-09

# Euclid specifications
sigma_epsilon  = 0.3
n_g = 30 #arcmin^-2. Surface density of galaxies
A_survey = 15000 #deg^2
N_z= 10 #number of redshift bins
ng_i = n_g/N_z # Surface density of galaxies per bin

In [3]:
# Matter power spectrum
P = 'c:/Users/MICROSOSFT/Documents/Investigacion 1/fiducial_eps_0/Pnonlin-zk.txt'
P_deltas = pd.read_csv(P, header=None, delimiter=' ')  # Cambia ' ' por el delimitador correcto

P_array_2d = P_deltas.values
P_list = P_deltas.values.flatten().tolist()
P_array_1d = np.array(P_list) # DONT USE

# Redshift
red = 'c:/Users/MICROSOSFT/Documents/Investigacion 1/fiducial_eps_0/z_values_list.txt'
z = pd.read_csv(red, header=None, delimiter=' ')  # Cambia ' ' por el delimitador correcto

z_list = z.values.flatten().tolist()
z_array = np.array(z_list)

# k values
kas = 'c:/Users/MICROSOSFT/Documents/Investigacion 1/fiducial_eps_0/k_values_list.txt'
k = pd.read_csv(kas, header=None, delimiter=' ')  # Cambia ' ' por el delimitador correcto

k_list = k.values.flatten().tolist()
k_array = np.array(k_list)

In [4]:
Omega_c0 = 0.2685628338348412

Omega_nu0 = 0.00143717

Omega_b0 = 0.05

Omega_m0 = Omega_c0 + Omega_nu0 + Omega_b0

Omega_DE0 = Omega_lambda = (1-Omega_m0)

w0= -1

wa = 0

Omega_K0 = 1 - (Omega_m0 + Omega_DE0)

h = 0.67

c = 299792

sigma8 = 0.816

ns = 0.96

gamma = 6/11

In [5]:
## Derivada con respecto a ns y sigma8

# Matter power spectrum_plus_ns
P_pl_ns_data = 'c:/Users/MICROSOSFT/Documents/Investigacion 1/ns_pl_eps_1p0E-2/Pnonlin-zk.txt'
P_deltas_pl_ns = pd.read_csv(P_pl_ns_data, header=None, delimiter=' ')  

P_pl_ns = P_deltas_pl_ns.values

# Matter power spectrum_minor_ns
P_mn_ns_data = 'c:/Users/MICROSOSFT/Documents/Investigacion 1/ns_mn_eps_1p0E-2/Pnonlin-zk.txt'
P_deltas_mn_ns = pd.read_csv(P_mn_ns_data, header=None, delimiter=' ')  

P_mn_ns = P_deltas_mn_ns.values

# Matter power spectrum_plus_sigma8
P_pl_sig_data = 'c:/Users/MICROSOSFT/Documents/Investigacion 1/s8_pl_eps_1p0E-2/Pnonlin-zk.txt'
P_deltas_pl_sig = pd.read_csv(P_pl_sig_data, header=None, delimiter=' ')  

P_pl_sig = P_deltas_pl_sig.values

# Matter power spectrum_minor_sigma8
P_mn_sig_data = 'c:/Users/MICROSOSFT/Documents/Investigacion 1/s8_mn_eps_1p0E-2/Pnonlin-zk.txt'
P_deltas_mn_sig = pd.read_csv(P_mn_sig_data, header=None, delimiter=' ')  

P_mn_sig = P_deltas_mn_sig.values

In [6]:
der_P_ns = (P_pl_ns - P_mn_ns)/(2*0.01*ns)

der_P_sigma8 = (P_pl_sig - P_mn_sig)/(2*0.01*sigma8)

In [7]:
class ClassIntegration:
    def __init__(self, params):
        self.z = params['z']

    def E2(self, z, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma):
        Oomega_m0 =  (Omega_c0 + Omega_nu0 + Omega_b0) 
        Omega_k0 = 1 - (Oomega_m0 + Omega_DE0) # If it is 0, the error of Omega_m0 increase its difference with respect to the article
        radicando = ((2*Oomega_m0) - Omega_m0) * (1 + z)**3 + (Omega_DE0 * (1 + z)**(3 * (1 + wa + w0)) * np.exp(-3 * wa * (z / (1 + z)))) + (Omega_k0) * (1 + z)**2
        return np.sqrt(radicando)

    def inverse_E2(self, z, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma):
        return 1 / self.E2(z, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma)

    def n_t(self, z):
        z_m, z_0 = 0.9, 0.9 / np.sqrt(2)
        return ((z / z_0)**2) * np.exp(-(z / z_0)**(3 / 2))

    def p_ph(self, z_p, z):
        def gauss(c, z0, s, z, zp):
            return (1 / (np.sqrt(2 * np.pi) * s * (1 + z))) * np.exp(-0.5 * ((z - (c * zp) - z0) / (s * (1 + z)))**2)
        return (1 - 0.1) * gauss(1, 0, 0.05, z, z_p) + 0.1 * gauss(1, 0.1, 0.05, z, z_p)

    def r(self, z, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma):
        c = 299792
        H_0 = 100 * h
        z_prime = np.linspace(0, z, 30)
        delta = z / len(z_prime)
        integrand = self.inverse_E2(z_prime, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma) * delta
        return np.sum(integrand) * (c / H_0)

    def n_i_try(self, i, z):
        z_bins = [0.001, 0.42, 0.56, 0.68, 0.79, 0.9, 1.02, 1.15, 1.32, 1.58, 2.5]
        denominators = np.array([0.04599087, 0.04048852, 0.04096115, 0.03951212, 0.03886145, 0.03944441, 0.03751183, 0.03950185, 0.04042198, 0.03827518])

        def numerator_n_i(i, z):
            z_prime = np.linspace(z_bins[i], z_bins[i + 1], 20)
            delta = (z_bins[i + 1] - z_bins[i]) / len(z_prime)
            multiplication_array = self.n_t(z) * self.p_ph(z_prime, z)
            return np.sum(multiplication_array * delta)

        return numerator_n_i(i, z) / denominators[i]

    def Window2(self, i, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma):
        H_0 = 100 * h
        result = []
        for z in self.z:
            z_max = 2.5
            z_prime = np.linspace(z, z_max, 20)
            delta = (z_max - z) / len(z_prime)
            r_true = self.r(z, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma)
            n_array = np.array([self.n_i_try(i, zs) for zs in z_prime])
            r_array = np.array([self.r(zs, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma) for zs in z_prime])
            integrand = n_array * (1 - ((r_true / c / H_0) / (r_array / c / H_0))) * delta
            result.append(np.sum(integrand))
        return np.array(result)

In [8]:
class CosmicShear:
    def __init__(self, cosmic_paramss):
        self.z = cosmic_paramss['z']
        self.l = cosmic_paramss['l']
        self.universe = cosmic_paramss['type']

    def E2(self, z, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma):
        Oomega_m0 =  (Omega_c0 + Omega_nu0 + Omega_b0) 
        Omega_k0 = 1 - (Oomega_m0 + Omega_DE0) # If it is 0, the error of Omega_m0 increase its difference with respect to the article
        radicando = ((2*Oomega_m0) - Omega_m0) * (1 + z)**3 + (Omega_DE0 * (1 + z)**(3 * (1 + wa + w0)) * np.exp(-3 * wa * (z / (1 + z)))) + (Omega_k0) * (1 + z)**2
        return np.sqrt(radicando)

    def inverse_E2(self, z, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma):
        return 1 / self.E2(z, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma)

    def r(self, z, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma):
        c = 299792
        H_0 = 100 * h
        z_prime = np.linspace(0, z, 30)
        delta = z / len(z_prime)
        integrand = self.inverse_E2(z_prime, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma) * delta
        return np.sum(integrand) * (c / H_0)

    def SN(self, i, j):
        return (0.3**2) / 35454308.58 if i == j else 0

    def D(self, z, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma):
        z_prime = np.linspace(0, z, 30)
        E_array = self.E2(z_prime, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma)
        Omega_m = (Omega_m0 * (1 + z_prime)**3) / (E_array**2)
        delta = z / len(z_prime)
        integral = np.sum((Omega_m**gamma / (1 + z_prime)) * delta)
        return np.exp(-integral)

    interp_func = RectBivariateSpline(z_array, np.log10(k_array), np.log10(P_array_2d))
    interp_func_ns = RectBivariateSpline(z_array, np.log10(k_array), der_P_ns)
    interp_func_sig = RectBivariateSpline(z_array, np.log10(k_array), der_P_sigma8)

    def PK(self, z, k):
        return 10**(self.interp_func(z, np.log10(k), grid=False))

    def der_PK_sig(self, z, k):
        return self.interp_func_sig(z, np.log10(k), grid=False)

    def der_PK_ns(self, z, k):
        return self.interp_func_ns(z, np.log10(k), grid=False)
    
    def PPS(self, z, l, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma):
        k = ((10**l + 0.5) / self.r(z, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma))
        P = float(self.PK(z, k))
        if self.universe == 'standard':
            return P
        else:
            D_0 = self.D(0, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma)
            return P*((self.D(z, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma)/D_0)**2)

    def der_PPS_ns(self, z, l, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma):
        k = ((10**l + 0.5)/self.r(z, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma))
        P = float(self.der_PK_ns(z, k))
        if self.universe == 'standard':
            return P
        else:
            D_0 = self.D(0, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma)
            return P*((self.D(z, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma)/D_0)**2)

    def der_PPS_sig(self, z, l, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma):
        k = ((10**l + 0.5)/self.r(z, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma))
        P = float(self.der_PK_sig(z, k))
        if self.universe == 'standard':
            return P
        else:
            D_0 = self.D(0, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma)
            return P*((self.D(z, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma)/D_0)**2)
    
    def Cosmic_Shear_l3(self, i ,j, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma):
        Aia, Cia, H_0, z_max, z_min = 1.72, 0.0134, 100 * h, 2.5, 0.001
        c = 299792
        z_prime, delta = self.z, (z_max - z_min) / len(self.z)
        SNs = self.SN(i, j)

        params = {'z': z_prime}
        
        A = ClassIntegration(params)

        result = []

        D_array = np.array([self.D(zs, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma) for zs in z_prime])
        E_array = self.E2(z_prime, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma)
        Wi = np.array(A.Window2(i, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma))
        n_i_array = np.array([A.n_i_try(i, zs) for zs in z_prime])
        Wj = np.array(A.Window2(j, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma))
        n_j_array = np.array([A.n_i_try(j, zs) for zs in z_prime])
        r_array = np.array([self.r(zs, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma) for zs in z_prime])

        operador1 = ((1.5 * Omega_m0 * (1+z_prime)) **2) * (H_0 / c) ** 3
        operador2 = 1.5* Omega_m0 * (1+z_prime) * (H_0 / c) ** 3
        operador3 =  (H_0 / c)**3

        K_gg = operador1 * (Wi * Wj / (E_array))
        K_Ig = operador2 * ((n_i_array * Wj) + (n_j_array * Wi)) / (r_array / c / H_0)
        K_II = operador3 * (n_i_array * n_j_array * E_array) / ((r_array / c / H_0) **2)
        
        for ls in self.l:

            P_gg = np.array([self.PPS(z_primes, ls, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma) for z_primes in z_prime])
            P_Ig = (-Aia * Cia * Omega_m0 / D_array) * np.array([self.PPS(z_primes, ls, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma)  for z_primes in z_prime])
            P_II = ((-Aia * Cia * Omega_m0 / D_array) ** 2) * np.array([self.PPS(z_primes, ls, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma)  for z_primes in z_prime])

            integrand = ((K_gg * P_gg) + (K_Ig * P_Ig) + (K_II * P_II)) * float(delta)
            integral =  np.sum(integrand)
            integral_final = integral + SNs

            result.append(integral_final)

        return np.array(result)
    
    def Der_C_ns(self, i ,j, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma): 
        Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma = Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma
        Aia, Cia, H_0, z_max, z_min = 1.72, 0.0134, 100 * h, 2.5, 0.001
        z_prime, delta = self.z, (z_max - z_min) / len(self.z)
        SNs = self.SN(i, j)

        params = {'z': z_prime}
        
        A = ClassIntegration(params)

        result = []

        D_array = np.array([self.D(zs, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma) for zs in z_prime])
        E_array = self.E2(z_prime, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma)
        Wi = np.array(A.Window2(i, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma))
        n_i_array = np.array([A.n_i_try(i, zs) for zs in z_prime])
        Wj = np.array(A.Window2(j, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma))
        n_j_array = np.array([A.n_i_try(j, zs) for zs in z_prime])
        r_array = np.array([self.r(zs, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma) for zs in z_prime])

        operador1 = ((1.5 * Omega_m0 * (1+z_prime)) **2) * (H_0 / c) ** 3
        operador2 = 1.5* Omega_m0 * (1+z_prime) * (H_0 / c) ** 3
        operador3 =  (H_0 / c)**3

        K_gg = operador1 * (Wi * Wj / (E_array))
        K_Ig = operador2 * ((n_i_array * Wj) + (n_j_array * Wi)) / (r_array / c / H_0)
        K_II = operador3 * (n_i_array * n_j_array * E_array) / ((r_array / c / H_0) **2)
        
        for ls in self.l:

            P_gg = np.array([self.der_PPS_ns(z_primes, ls, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma) for z_primes in z_prime])
            P_Ig = (-Aia * Cia * Omega_m0 / D_array) * np.array([self.der_PPS_ns(z_primes, ls, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma)  for z_primes in z_prime])
            P_II = ((-Aia * Cia * Omega_m0 / D_array) ** 2) * np.array([self.der_PPS_ns(z_primes, ls, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma)  for z_primes in z_prime])

            integrand = ((K_gg * P_gg) + (K_Ig * P_Ig) + (K_II * P_II)) * float(delta) 
            integral =  np.sum(integrand)
            integral_final = integral + SNs

            result.append(integral_final)

        return np.array(result)
    
    def Der_C_sig(self, i ,j, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma):
        Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma = Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma
        Aia, Cia, H_0, z_max, z_min = 1.72, 0.0134, 100 * h, 2.5, 0.001
        z_prime, delta = self.z, (z_max - z_min) / len(self.z)
        SNs = self.SN(i, j)

        param = {'z': z_prime}
        
        A = ClassIntegration(param)

        result = []

        D_array = np.array([self.D(zs, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma) for zs in z_prime])
        E_array = self.E2(z_prime, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma)
        Wi = np.array(A.Window2(i, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma))
        n_i_array = np.array([A.n_i_try(i, zs) for zs in z_prime])
        Wj = np.array(A.Window2(j, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma))
        n_j_array = np.array([A.n_i_try(j, zs) for zs in z_prime])
        r_array = np.array([self.r(zs, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma) for zs in z_prime])

        operador1 = ((1.5 * Omega_m0 * (1+z_prime)) **2) * (H_0 / c) ** 3
        operador2 = 1.5* Omega_m0 * (1+z_prime) * (H_0 / c) ** 3
        operador3 =  (H_0 / c)**3

        K_gg = operador1 * (Wi * Wj / (E_array))
        K_Ig = operador2 * ((n_i_array * Wj) + (n_j_array * Wi)) / (r_array / c / H_0)
        K_II = operador3 * (n_i_array * n_j_array * E_array) / ((r_array / c / H_0) **2)
        
        for ls in self.l:

            P_gg = np.array([self.der_PPS_sig(z_primes, ls, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma) for z_primes in z_prime])
            P_Ig = (-Aia * Cia * Omega_m0 / D_array) * np.array([self.der_PPS_sig(z_primes, ls, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma)  for z_primes in z_prime])
            P_II = ((-Aia * Cia * Omega_m0 / D_array) ** 2) * np.array([self.der_PPS_sig(z_primes, ls, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma)  for z_primes in z_prime])

            integrand = ((K_gg * P_gg) + (K_Ig * P_Ig) + (K_II * P_II)) * float(delta) 
            integral =  np.sum(integrand)
            integral_final = integral + SNs

            result.append(integral_final)

        return np.array(result)
    
    def Der_C_Omega(self, i, j, epsilon, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma):
        C_plus = self.Cosmic_Shear_l3(i, j, Omega_m0*(1+epsilon), h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma)
        C_minus = self.Cosmic_Shear_l3(i, j, Omega_m0*(1-epsilon), h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma)

        return (C_plus - C_minus)/(epsilon*2*Omega_m0)

    def Der_C_h(self, i, j, epsilon, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma):
        C_plus = self.Cosmic_Shear_l3(i, j, Omega_m0, h*(1+epsilon), Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma)
        C_minus = self.Cosmic_Shear_l3(i, j, Omega_m0, h*(1-epsilon), Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma)

        return (C_plus - C_minus)/(epsilon*2*h)

    def Der_C_Omega_b(self, i, j, epsilon, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma):
        C_plus = self.Cosmic_Shear_l3(i, j, Omega_m0, h, Omega_b0*(1+epsilon), Omega_DE0, w0, wa, ns, sigma8, gamma)
        C_minus = self.Cosmic_Shear_l3(i, j, Omega_m0, h, Omega_b0*(1-epsilon), Omega_DE0, w0, wa, ns, sigma8, gamma)

        return (C_plus - C_minus)/(epsilon*2*Omega_b0)

    def Der_C_Omega_DE(self, i, j, epsilon, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma):
        C_plus = self.Cosmic_Shear_l3(i, j, Omega_m0, h, Omega_b0, Omega_DE0*(1+epsilon), w0, wa, ns, sigma8, gamma)
        C_minus = self.Cosmic_Shear_l3(i, j, Omega_m0, h, Omega_b0, Omega_DE0*(1-epsilon), w0, wa, ns, sigma8, gamma)

        return (C_plus - C_minus)/(epsilon*2*Omega_DE0)

    def Der_C_w0(self, i, j, epsilon, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma):
        C_plus = self.Cosmic_Shear_l3(i, j, Omega_m0, h, Omega_b0, Omega_DE0, w0*(1+epsilon), wa, ns, sigma8, gamma)
        C_minus = self.Cosmic_Shear_l3(i, j, Omega_m0, h, Omega_b0, Omega_DE0, w0*(1-epsilon), wa, ns, sigma8, gamma)

        A = (C_plus - C_minus)/(epsilon*2*w0)

        return A

    def Der_C_wa(self, i, j, epsilon, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma):
        C_plus = self.Cosmic_Shear_l3(i, j, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa + epsilon, ns, sigma8, gamma)
        C_minus = self.Cosmic_Shear_l3(i, j, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa - epsilon, ns, sigma8, gamma)

        return (C_plus - C_minus)/(epsilon*2)
    
    def Der_C_gamma(self, i, j, epsilon, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma):
        '''
        Integra las funciones derivadas
        '''
        
        C_plus = self.Cosmic_Shear_l3(i, j, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma*(1+epsilon))
        C_minus = self.Cosmic_Shear_l3(i, j, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma*(1-epsilon))

        return (C_plus - C_minus)/(epsilon*2*gamma)

    ##
        
    def Gran_Derivative(self, i, j, epsilon, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma, parametro1):
        if parametro1 == 'Om_m':
            return self.Der_C_Omega(i, j, epsilon, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma)
        elif parametro1 == 'Om_b':
            return self.Der_C_Omega_b(i, j, epsilon, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma)
        elif parametro1 == 'h':
            return self.Der_C_h(i, j, epsilon, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma)
        elif parametro1 == 'Om_DE':
            return self.Der_C_Omega_DE(i, j, epsilon, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma)
        elif parametro1 == 'w0':
            return self.Der_C_w0(i, j, epsilon, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma)
        elif parametro1 == 'wa':
            return self.Der_C_wa(i, j, epsilon, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma)
        elif parametro1 == 'ns':
            return self.Der_C_ns(i ,j, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma)
        elif parametro1 == 'sigma8':
            return self.Der_C_sig(i ,j, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma)
        elif parametro1 == 'gamma':
            return self.Der_C_gamma(i ,j, epsilon, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma)

In [102]:
class Fisher:
    '''
    Calculate Fisher Matrix
    '''
    def __init__(self, params):
        self.num = params['num_params']
        self.universes = params['type']
        self.model = params['model']

    def trace(self, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma):
        F = np.zeros((self.num, self.num))
        L_array = np.log10(np.logspace(np.log10(10), np.log10(1500), 100))
        cosmic_params = {'l': L_array , 'z': np.linspace(0.001, 2.5, 10), 'type': self.universes}
        A = CosmicShear(cosmic_params)
        
        f_sky = 0.3636
        C = np.zeros((10, 10), dtype=object)
        dC_dq_Om_m_matrix = np.zeros((10, 10), dtype=object)
        dC_dq_h_matrix = np.zeros((10, 10), dtype=object)
        dC_dq_Om_b_matrix = np.zeros((10, 10), dtype=object)
        dC_dq_Om_DE_matrix = np.zeros((10, 10), dtype=object)
        dC_dq_w0_matrix = np.zeros((10, 10), dtype=object)
        dC_dq_wa_matrix = np.zeros((10, 10), dtype=object)
        dC_dq_ns_matrix = np.zeros((10, 10), dtype=object)
        dC_dq_sig_matrix = np.zeros((10, 10), dtype=object)
        dC_dq_gamma_matrix = np.zeros((10, 10), dtype=object)
        
        for i in range(10):
            for j in range(i, 10):
                logging.info(f"Calculated pair: {(i, j)}")
                C[i, j] = C[j, i] = A.Cosmic_Shear_l3(i, j, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma)
                dC_dq_Om_m_matrix[i, j] = dC_dq_Om_m_matrix[j, i] = A.Gran_Derivative(i, j, 0.01, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma, 'Om_m')
                dC_dq_h_matrix[i, j] = dC_dq_h_matrix[j, i] = A.Gran_Derivative(i, j, 0.01, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma, 'h')
                dC_dq_Om_b_matrix[i, j] = dC_dq_Om_b_matrix[j, i] = A.Gran_Derivative(i, j, 0.01, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma, 'Om_b')
                dC_dq_Om_DE_matrix[i, j] = dC_dq_Om_DE_matrix[j, i] =  A.Gran_Derivative(i, j, 0.01, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma, 'Om_DE')
                dC_dq_w0_matrix[i, j] = dC_dq_w0_matrix[j, i] = A.Gran_Derivative(i, j, 0.01, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma, 'w0')
                dC_dq_wa_matrix[i, j] = dC_dq_wa_matrix[j, i] = A.Gran_Derivative(i, j, 0.01, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma, 'wa')
                dC_dq_ns_matrix[i, j] =  dC_dq_ns_matrix[j, i]  = A.Gran_Derivative(i, j, 0.01, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma, 'ns')
                dC_dq_sig_matrix[i, j] = dC_dq_sig_matrix[j, i] = A.Gran_Derivative(i, j, 0.01, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma, 'sigma8')
                dC_dq_gamma_matrix[i, j] = dC_dq_gamma_matrix[j, i] = A.Gran_Derivative(i, j, 0.01, Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma, 'gamma')

        C_matrix = np.array(C.tolist(), dtype=float)
        C_matrix_2d = np.squeeze(C_matrix)

        dC_dq_Om_m_matrix = np.array(dC_dq_Om_m_matrix.tolist(), dtype=float)
        dC_dq_h_matrix = np.array(dC_dq_h_matrix.tolist(), dtype=float)
        dC_dq_Om_b_matrix = np.array(dC_dq_Om_b_matrix.tolist(), dtype=float)
        dC_dq_Om_DE_matrix = np.array(dC_dq_Om_DE_matrix.tolist(), dtype=float)
        dC_dq_w0_matrix = np.array(dC_dq_w0_matrix.tolist(), dtype=float)
        dC_dq_wa_matrix = np.array(dC_dq_wa_matrix.tolist(), dtype=float)
        dC_dq_ns_matrix = np.array(dC_dq_ns_matrix.tolist(), dtype=float)
        dC_dq_sig_matrix = np.array(dC_dq_sig_matrix.tolist(), dtype=float)
        dC_dq_gamma_matrix = np.array(dC_dq_gamma_matrix.tolist(), dtype=float)

        dC_dq_Om_m_matrix_2d = np.squeeze(dC_dq_Om_m_matrix)
        dC_dq_h_matrix_2d = np.squeeze(dC_dq_h_matrix)
        dC_dq_Om_b_matrix_2d = np.squeeze(dC_dq_Om_b_matrix)
        dC_dq_Om_DE_matrix_2d = np.squeeze(dC_dq_Om_DE_matrix)
        dC_dq_w0_matrix_2d = np.squeeze(dC_dq_w0_matrix)
        dC_dq_wa_matrix_2d = np.squeeze(dC_dq_wa_matrix)
        dC_dq_ns_matrix_2d = np.squeeze(dC_dq_ns_matrix)
        dC_dq_sig_matrix_2d = np.squeeze(dC_dq_sig_matrix)
        dC_dq_gamma_matrix_2d = np.squeeze(dC_dq_gamma_matrix)
        
        for i, l in enumerate(L_array):

            L_parameter = ((2*(10**(l)) + 1)/2)
            C_inv = np.linalg.inv(C_matrix_2d[:, :, i])

            # OMEGA_M 
            Om_m_Om_m = np.trace(dC_dq_Om_m_matrix_2d[:, :, i] @ C_inv @ dC_dq_Om_m_matrix_2d[:, :, i] @ C_inv)
            Om_m_h = np.trace(dC_dq_Om_m_matrix_2d[:, :, i] @ C_inv @ dC_dq_h_matrix_2d[:, :, i] @ C_inv)
            Om_m_Om_b = np.trace(dC_dq_Om_m_matrix_2d[:, :, i] @ C_inv @ dC_dq_Om_b_matrix_2d[:, :, i] @ C_inv)
            Om_m_Om_DE = np.trace(dC_dq_Om_m_matrix_2d[:, :, i] @ C_inv @ dC_dq_Om_DE_matrix_2d[:, :, i] @ C_inv)
            Om_m_w0 = np.trace(dC_dq_Om_m_matrix_2d[:, :, i] @ C_inv @ dC_dq_w0_matrix_2d[:, :, i] @ C_inv)
            Om_m_wa = np.trace(dC_dq_Om_m_matrix_2d[:, :, i] @ C_inv @ dC_dq_wa_matrix_2d[:, :, i] @ C_inv)
            Om_m_ns = np.trace(dC_dq_Om_m_matrix_2d[:, :, i] @ C_inv @ dC_dq_ns_matrix_2d[:, :, i] @ C_inv)
            Om_m_sig = np.trace(dC_dq_Om_m_matrix_2d[:, :, i] @ C_inv @ dC_dq_sig_matrix_2d[:, :, i] @ C_inv)
            Om_m_gamma = np.trace(dC_dq_Om_m_matrix_2d[:, :, i] @ C_inv @ dC_dq_gamma_matrix_2d[:, :, i] @ C_inv)


            # H
            h_h = np.trace(dC_dq_h_matrix_2d[:, :, i] @ C_inv @ dC_dq_h_matrix_2d[:, :, i] @ C_inv)
            h_Om_b = np.trace(dC_dq_h_matrix_2d[:, :, i] @ C_inv @ dC_dq_Om_b_matrix_2d[:, :, i] @ C_inv)
            h_Om_DE = np.trace(dC_dq_h_matrix_2d[:, :, i] @ C_inv @ dC_dq_Om_DE_matrix_2d[:, :, i] @ C_inv)
            h_w0 = np.trace(dC_dq_h_matrix_2d[:, :, i] @ C_inv @ dC_dq_w0_matrix_2d[:, :, i] @ C_inv)
            h_wa = np.trace(dC_dq_h_matrix_2d[:, :, i] @ C_inv @ dC_dq_wa_matrix_2d[:, :, i] @ C_inv)
            h_ns = np.trace(dC_dq_h_matrix_2d[:, :, i] @ C_inv @ dC_dq_ns_matrix_2d[:, :, i] @ C_inv)
            h_sig = np.trace(dC_dq_h_matrix_2d[:, :, i] @ C_inv @ dC_dq_sig_matrix_2d[:, :, i] @ C_inv)
            h_gamma = np.trace(dC_dq_h_matrix_2d[:, :, i] @ C_inv @ dC_dq_gamma_matrix_2d[:, :, i] @ C_inv)

            # OMEGA_B
            Om_b_Om_b = np.trace(dC_dq_Om_b_matrix_2d[:, :, i] @ C_inv @ dC_dq_Om_b_matrix_2d[:, :, i] @ C_inv)
            Om_b_Om_DE = np.trace(dC_dq_Om_b_matrix_2d[:, :, i] @ C_inv @ dC_dq_Om_DE_matrix_2d[:, :, i] @ C_inv)
            Om_b_w0 = np.trace(dC_dq_Om_b_matrix_2d[:, :, i] @ C_inv @ dC_dq_w0_matrix_2d[:, :, i] @ C_inv)
            Om_b_wa = np.trace(dC_dq_Om_b_matrix_2d[:, :, i] @ C_inv @ dC_dq_wa_matrix_2d[:, :, i] @ C_inv)
            Om_b_ns = np.trace(dC_dq_Om_b_matrix_2d[:, :, i] @ C_inv @ dC_dq_ns_matrix_2d[:, :, i] @ C_inv)
            Om_b_sig = np.trace(dC_dq_Om_b_matrix_2d[:, :, i] @ C_inv @ dC_dq_sig_matrix_2d[:, :, i] @ C_inv)
            Om_b_gamma = np.trace(dC_dq_Om_b_matrix_2d[:, :, i] @ C_inv @ dC_dq_gamma_matrix_2d[:, :, i] @ C_inv)

            # OMEGA_DE
            Om_DE_Om_DE = np.trace(dC_dq_Om_DE_matrix_2d[:, :, i] @ C_inv @ dC_dq_Om_DE_matrix_2d[:, :, i] @ C_inv)
            Om_DE_w0 = np.trace(dC_dq_Om_DE_matrix_2d[:, :, i] @ C_inv @ dC_dq_w0_matrix_2d[:, :, i] @ C_inv)
            Om_DE_wa = np.trace(dC_dq_Om_DE_matrix_2d[:, :, i] @ C_inv @ dC_dq_wa_matrix_2d[:, :, i] @ C_inv)
            Om_DE_ns = np.trace(dC_dq_Om_DE_matrix_2d[:, :, i] @ C_inv @ dC_dq_ns_matrix_2d[:, :, i] @ C_inv)
            Om_DE_sig = np.trace(dC_dq_Om_DE_matrix_2d[:, :, i] @ C_inv @ dC_dq_sig_matrix_2d[:, :, i] @ C_inv)
            Om_DE_gamma = np.trace(dC_dq_Om_DE_matrix_2d[:, :, i] @ C_inv @ dC_dq_gamma_matrix_2d[:, :, i] @ C_inv)

            # W0
            w0_w0 = np.trace(dC_dq_w0_matrix_2d[:, :, i] @ C_inv @ dC_dq_w0_matrix_2d[:, :, i] @ C_inv)
            w0_wa = np.trace(dC_dq_w0_matrix_2d[:, :, i] @ C_inv @ dC_dq_wa_matrix_2d[:, :, i] @ C_inv)
            w0_ns = np.trace(dC_dq_w0_matrix_2d[:, :, i] @ C_inv @ dC_dq_ns_matrix_2d[:, :, i] @ C_inv)
            w0_sig = np.trace(dC_dq_w0_matrix_2d[:, :, i] @ C_inv @ dC_dq_sig_matrix_2d[:, :, i] @ C_inv)
            w0_gamma = np.trace(dC_dq_w0_matrix_2d[:, :, i] @ C_inv @ dC_dq_gamma_matrix_2d[:, :, i] @ C_inv)

            # WA
            wa_wa = np.trace(dC_dq_wa_matrix_2d[:, :, i] @ C_inv @ dC_dq_wa_matrix_2d[:, :, i] @ C_inv)
            wa_ns = np.trace(dC_dq_wa_matrix_2d[:, :, i] @ C_inv @ dC_dq_ns_matrix_2d[:, :, i] @ C_inv)
            wa_sig = np.trace(dC_dq_wa_matrix_2d[:, :, i] @ C_inv @ dC_dq_sig_matrix_2d[:, :, i] @ C_inv) 
            wa_gamma = np.trace(dC_dq_wa_matrix_2d[:, :, i] @ C_inv @ dC_dq_gamma_matrix_2d[:, :, i] @ C_inv)           

            # NS
            ns_ns = np.trace(dC_dq_ns_matrix_2d[:, :, i] @ C_inv @ dC_dq_ns_matrix_2d[:, :, i] @ C_inv)
            ns_sig = np.trace(dC_dq_ns_matrix_2d[:, :, i] @ C_inv @ dC_dq_sig_matrix_2d[:, :, i] @ C_inv)
            ns_gamma = np.trace(dC_dq_ns_matrix_2d[:, :, i] @ C_inv @ dC_dq_gamma_matrix_2d[:, :, i] @ C_inv)

            # SIGMA8
            sig_sig = np.trace(dC_dq_sig_matrix_2d[:, :, i] @ C_inv @ dC_dq_sig_matrix_2d[:, :, i] @ C_inv) 
            sig_gamma = np.trace(dC_dq_sig_matrix_2d[:, :, i] @ C_inv @ dC_dq_gamma_matrix_2d[:, :, i] @ C_inv)   

            # GAMMA
            gamma_gamma = np.trace(dC_dq_gamma_matrix_2d[:, :, i] @ C_inv @ dC_dq_gamma_matrix_2d[:, :, i] @ C_inv)  

            if self.model == 'ACDM_flat':
                coeficientes = {
                    (0, 0): Om_m_Om_m, (0, 1): Om_m_h, (0, 2): Om_m_Om_b, (0, 3): Om_m_ns, (0, 4): Om_m_sig, 
                    (1, 1): h_h, (1, 2): h_Om_b, (1, 3): h_ns, (1, 4): h_sig, 
                    (2, 2): Om_b_Om_b, (2, 3): Om_b_ns, (2, 4): Om_b_sig,
                    (3, 3): ns_ns, (3, 4): ns_sig,
                    (4, 4): sig_sig
                }

                for (i, j), coef in coeficientes.items():
                    F[i, j] += L_parameter * coef
            
            if self.model == 'ACDM_non_flat':
                coeficientes = {
                    (0, 0): Om_m_Om_m, (0, 1): Om_m_h, (0, 2): Om_m_Om_b, (0, 3):Om_m_Om_DE, (0, 4): Om_m_ns, (0, 5): Om_m_sig, 
                    (1, 1): h_h, (1, 2): h_Om_b, (1, 3): h_Om_DE, (1, 4): h_ns, (1, 5): h_sig, 
                    (2, 2): Om_b_Om_b, (2, 3): Om_b_Om_DE, (2, 4): Om_b_ns, (2, 5): Om_b_sig,
                    (3, 3): Om_DE_Om_DE, (3, 4): Om_DE_ns, (3, 5): Om_DE_sig, 
                    (4, 4): ns_ns, (4, 5): ns_sig,
                    (5, 5): sig_sig
                }

                for (i, j), coef in coeficientes.items():
                    F[i, j] += L_parameter * coef

            if self.model == 'non_ACDM_flat':
                coeficientes = {
                    (0, 0): Om_m_Om_m, (0, 1): Om_m_h, (0, 2): Om_m_Om_b, (0, 3): Om_m_w0, (0, 4): Om_m_wa, (0, 5): Om_m_ns, (0, 6): Om_m_sig,
                    (1, 1): h_h, (1, 2): h_Om_b, (1, 3): h_w0, (1, 4): h_wa, (1, 5): h_ns, (1, 6): h_sig,
                    (2, 2): Om_b_Om_b, (2, 3): Om_b_w0, (2, 4): Om_b_wa, (2, 5): Om_b_ns, (2, 6): Om_b_sig,  
                    (3, 3): w0_w0, (3, 4): w0_wa, (3, 5): w0_ns, (3, 6): w0_sig,
                    (4, 4): wa_wa, (4, 5): wa_ns, (4, 6): wa_sig,
                    (5, 5): ns_ns, (5, 6): ns_sig,
                    (6, 6): sig_sig
                }

                for (i, j), coef in coeficientes.items():
                    F[i, j] += L_parameter * coef

            if self.model == 'non_ACDM_non_flat':
                coeficientes = {
                    (0, 0): Om_m_Om_m, (0, 1): Om_m_h, (0, 2): Om_m_Om_b, (0, 3): Om_m_Om_DE, (0, 4): Om_m_w0, (0, 5): Om_m_wa, (0, 6): Om_m_ns, (0, 7): Om_m_sig,
                    (1, 1): h_h, (1, 2): h_Om_b, (1, 3): h_Om_DE, (1, 4): h_w0, (1, 5): h_wa, (1, 6): h_ns, (1, 7): h_sig,
                    (2, 2): Om_b_Om_b, (2, 3): Om_b_Om_DE, (2, 4): Om_b_w0, (2, 5): Om_b_wa, (2, 6): Om_b_ns, (2, 7): Om_b_sig,  
                    (3, 3): Om_DE_Om_DE, (3, 4): Om_DE_w0, (3, 5): Om_DE_wa, (3, 6): Om_DE_ns, (3, 7): Om_DE_sig,
                    (4, 4): w0_w0, (4, 5): w0_wa, (4, 6): w0_ns, (4, 7): w0_sig,
                    (5, 5): wa_wa, (5, 6): wa_ns, (5, 7): wa_sig,
                    (6, 6): ns_ns, (6, 7): ns_sig,
                    (7, 7): sig_sig
                }
                for (i, j), coef in coeficientes.items():
                    F[i, j] += L_parameter * coef

            if self.model == 'non_ACDM_non_flat_gamma':
                coeficientes = {
                    (0, 0): Om_m_Om_m, (0, 1): Om_m_h, (0, 2): Om_m_Om_b, (0, 3): Om_m_Om_DE, (0, 4): Om_m_w0, (0, 5): Om_m_wa, (0, 6): Om_m_ns, (0, 7): Om_m_sig, (0, 8): Om_m_gamma,
                    (1, 1): h_h, (1, 2): h_Om_b, (1, 3): h_Om_DE, (1, 4): h_w0, (1, 5): h_wa, (1, 6): h_ns, (1, 7): h_sig, (1, 8): h_gamma,
                    (2, 2): Om_b_Om_b, (2, 3): Om_b_Om_DE, (2, 4): Om_b_w0, (2, 5): Om_b_wa, (2, 6): Om_b_ns, (2, 7): Om_b_sig, (2, 8): Om_b_gamma,
                    (3, 3): Om_DE_Om_DE, (3, 4): Om_DE_w0, (3, 5): Om_DE_wa, (3, 6): Om_DE_ns, (3, 7): Om_DE_sig, (3, 8): Om_DE_gamma,
                    (4, 4): w0_w0, (4, 5): w0_wa, (4, 6): w0_ns, (4, 7): w0_sig, (4, 8): w0_gamma,
                    (5, 5): wa_wa, (5, 6): wa_ns, (5, 7): wa_sig, (5, 8): wa_gamma,
                    (6, 6): ns_ns, (6, 7): ns_sig, (6, 8): ns_gamma,
                    (7, 7): sig_sig, (7, 8): sig_gamma,
                    (8, 8): gamma_gamma
                }
                for (i, j), coef in coeficientes.items():
                    F[i, j] += L_parameter * coef

        Fisher = F + F.T - np.diag(F.diagonal())
        logging.info(f"Fisher Matrix: {Fisher}")

        return f_sky*Fisher

# $\Lambda CDM$ flat

Parameters:

1. $\Omega_{m,0}$

2. $h$

3. $\Omega_{b,0}$

4. $\Omega{DE,0}$

5. $n_s$

6. $\sigma_9$

In [103]:
params = {'num_params': 5, 'type': 'standard', 'model' : 'ACDM_flat'}

A = Fisher(params)

In [104]:
F = A.trace(Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma)

INFO:root:Calculated pair: (0, 0)


INFO:root:Calculated pair: (0, 1)
INFO:root:Calculated pair: (0, 2)
INFO:root:Calculated pair: (0, 3)
INFO:root:Calculated pair: (0, 4)
INFO:root:Calculated pair: (0, 5)
INFO:root:Calculated pair: (0, 6)
INFO:root:Calculated pair: (0, 7)
INFO:root:Calculated pair: (0, 8)
INFO:root:Calculated pair: (0, 9)
INFO:root:Calculated pair: (1, 1)
INFO:root:Calculated pair: (1, 2)
INFO:root:Calculated pair: (1, 3)
INFO:root:Calculated pair: (1, 4)
INFO:root:Calculated pair: (1, 5)
INFO:root:Calculated pair: (1, 6)
INFO:root:Calculated pair: (1, 7)
INFO:root:Calculated pair: (1, 8)
INFO:root:Calculated pair: (1, 9)
INFO:root:Calculated pair: (2, 2)
INFO:root:Calculated pair: (2, 3)
INFO:root:Calculated pair: (2, 4)
INFO:root:Calculated pair: (2, 5)
INFO:root:Calculated pair: (2, 6)
INFO:root:Calculated pair: (2, 7)
INFO:root:Calculated pair: (2, 8)
INFO:root:Calculated pair: (2, 9)
INFO:root:Calculated pair: (3, 3)
INFO:root:Calculated pair: (3, 4)
INFO:root:Calculated pair: (3, 5)
INFO:root:Calc

In [105]:
F

array([[4301663.92417452, 4792067.93060032,  547018.32110502,
        -275992.19416583, 1561444.0636529 ],
       [4792067.93060032, 5630038.78759374,  737712.48482231,
        -346573.7097149 , 1833699.80359352],
       [ 547018.32110502,  737712.48482231,  131746.93569986,
         -60019.57496907,  233043.32558601],
       [-275992.19416583, -346573.7097149 ,  -60019.57496907,
          84679.65524812,  -60248.34313754],
       [1561444.0636529 , 1833699.80359352,  233043.32558601,
         -60248.34313754,  648242.1948947 ]])

In [106]:
Cov = np.linalg.inv(F)

print('ERRORES')

print('Error Omega_m: ' + str(np.sqrt(Cov[0, 0])))
print('Error h: ' + str(np.sqrt(Cov[1, 1])))
print('Error Omega_b: ' + str(np.sqrt(Cov[2, 2])))
print('Error ns: ' + str(np.sqrt(Cov[3, 3])))
print('Error sigma8: ' + str(np.sqrt(Cov[4, 4])))

print(' ')

# Relative errors

print('RELATIVE ERRORS')

err_Om_m = np.sqrt(Cov[0, 0]) / Omega_m0
err_h = np.sqrt(Cov[1, 1]) / h
err_Om_b = np.sqrt(Cov[2, 2]) / Omega_b0
err_ns = np.sqrt(Cov[3, 3]) / ns
err_sig = np.sqrt(Cov[4, 4]) / sigma8

print('Error Relativo Omega_m: ' + str(err_Om_m))
print('Error Relativo h: ' + str(err_h))
print('Error Relativo Omega_b: ' + str(err_Om_b))
print('Error Relativo ns: ' + str(err_ns))
print('Error Relativo sigma8: ' + str(err_sig))

print(' ')

print('COMPARACION')
## Comparison with the paper (the numbers must be under 10)

S_Om_m = 0.018
S_h = 0.21
S_Om_b = 0.47
S_ns = 0.035
S_sig = 0.0087

def comparison(created, expected):
    return 100*np.abs(1 - (created/expected))

print('Comparison for Omega_m: ' + str(comparison(err_Om_m, S_Om_m)))
print('Comparison for h: ' + str(comparison(err_h, S_h)))
print('Comparison for Omega_b: ' + str(comparison(err_Om_b, S_Om_b)))
print('Comparison for ns: ' + str(comparison(err_ns, S_ns)))
print('Comparison for sigma8: ' + str(comparison(err_sig, S_sig)))

ERRORES
Error Omega_m: 0.005687514076042387
Error h: 0.008818786427501687
Error Omega_b: 0.014544529167889257
Error ns: 0.011561262818484727
Error sigma8: 0.012709519138071036
 
RELATIVE ERRORS
Error Relativo Omega_m: 0.017773481274637216
Error Relativo h: 0.013162367802241323
Error Relativo Omega_b: 0.2908905833577851
Error Relativo ns: 0.012042982102588258
Error Relativo sigma8: 0.01557539110057725
 
COMPARACION
Comparison for Omega_m: 1.2584373631265633
Comparison for h: 93.73220580845651
Comparison for Omega_b: 38.10838651962018
Comparison for ns: 65.5914797068907
Comparison for sigma8: 79.02748391468106


# $\Lambda CDM$ non-flat

Parameters:

1. $\Omega_{m,0}$

2. $h$

3. $\Omega_{b,0}$

4. $\Omega{DE,0}$

5. $n_s$

6. $\sigma_9$

In [107]:
params = {'num_params': 6, 'type': 'standard', 'model' : 'ACDM_non_flat'}

B = Fisher(params)

In [108]:
F_2 = B.trace(Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma)

INFO:root:Calculated pair: (0, 0)
INFO:root:Calculated pair: (0, 1)
INFO:root:Calculated pair: (0, 2)
INFO:root:Calculated pair: (0, 3)
INFO:root:Calculated pair: (0, 4)
INFO:root:Calculated pair: (0, 5)
INFO:root:Calculated pair: (0, 6)
INFO:root:Calculated pair: (0, 7)
INFO:root:Calculated pair: (0, 8)
INFO:root:Calculated pair: (0, 9)
INFO:root:Calculated pair: (1, 1)
INFO:root:Calculated pair: (1, 2)
INFO:root:Calculated pair: (1, 3)
INFO:root:Calculated pair: (1, 4)
INFO:root:Calculated pair: (1, 5)
INFO:root:Calculated pair: (1, 6)
INFO:root:Calculated pair: (1, 7)
INFO:root:Calculated pair: (1, 8)
INFO:root:Calculated pair: (1, 9)
INFO:root:Calculated pair: (2, 2)
INFO:root:Calculated pair: (2, 3)
INFO:root:Calculated pair: (2, 4)
INFO:root:Calculated pair: (2, 5)
INFO:root:Calculated pair: (2, 6)
INFO:root:Calculated pair: (2, 7)
INFO:root:Calculated pair: (2, 8)
INFO:root:Calculated pair: (2, 9)
INFO:root:Calculated pair: (3, 3)
INFO:root:Calculated pair: (3, 4)
INFO:root:Calc

In [109]:
F_2

array([[4301663.92417452, 4792067.93060032,  547018.32110502,
        -110629.2849992 , -275992.19416583, 1561444.0636529 ],
       [4792067.93060032, 5630038.78759374,  737712.48482231,
        -158018.20487562, -346573.7097149 , 1833699.80359352],
       [ 547018.32110502,  737712.48482231,  131746.93569986,
         -30665.29935843,  -60019.57496907,  233043.32558601],
       [-110629.2849992 , -158018.20487562,  -30665.29935843,
           7447.25637224,   12752.56080646,  -50441.2771906 ],
       [-275992.19416583, -346573.7097149 ,  -60019.57496907,
          12752.56080646,   84679.65524812,  -60248.34313754],
       [1561444.0636529 , 1833699.80359352,  233043.32558601,
         -50441.2771906 ,  -60248.34313754,  648242.1948947 ]])

In [110]:
Cov = np.linalg.inv(F_2)

print('ERRORES')

print('Error Omega_m: ' + str(np.sqrt(Cov[0, 0])))
print('Error h: ' + str(np.sqrt(Cov[1, 1])))
print('Error Omega_b: ' + str(np.sqrt(Cov[2, 2])))
print('Error Omega_DE: ' + str(np.sqrt(Cov[3, 3])))
print('Error ns: ' + str(np.sqrt(Cov[4, 4])))
print('Error sigma8: ' + str(np.sqrt(Cov[5, 5])))

print(' ')

# Relative errors

err_Om_m = np.sqrt(Cov[0, 0]) / Omega_m0
err_h = np.sqrt(Cov[1, 1]) / h
err_Om_b = np.sqrt(Cov[2, 2]) / Omega_b0
err_Om_DE = np.sqrt(Cov[3, 3]) / Omega_DE0
err_ns = np.sqrt(Cov[4, 4]) / ns
err_sig = np.sqrt(Cov[5, 5]) / sigma8

print('Error Relativo Omega_m: ' + str(err_Om_m))
print('Error Relativo h: ' + str(err_h))
print('Error Relativo Omega_b: ' + str(err_Om_b))
print('Error Relativo Omega_DE: ' + str(err_Om_DE))
print('Error Relativo ns: ' + str(err_ns))
print('Error Relativo sigma8: ' + str(err_sig))

print(' ')

print('COMPARACION')
## Comparison with the paper (the numbers must be under 10)

S_Om_m = 0.021
S_h = 0.21
S_Om_b = 0.47
S_Om_DE = 0.035
S_ns = 0.035
S_sig = 0.015

print('Comparison for Omega_m: ' + str(comparison(err_Om_m, S_Om_m)))
print('Comparison for h: ' + str(comparison(err_h, S_h)))
print('Comparison for Omega_b: ' + str(comparison(err_Om_b, S_Om_b)))
print('Comparison for Omega_DE: ' + str(comparison(err_Om_DE, S_Om_DE)))
print('Comparison for ns: ' + str(comparison(err_ns, S_ns)))
print('Comparison for sigma8: ' + str(comparison(err_sig, S_sig)))

ERRORES
Error Omega_m: 0.005895237743205239
Error h: 0.008885438807769899
Error Omega_b: 0.024764586869997444
Error Omega_DE: 0.08139611319281834
Error ns: 0.011631326473150936
Error sigma8: 0.0127098384520748
 
Error Relativo Omega_m: 0.018422617726741957
Error Relativo h: 0.013261848966820743
Error Relativo Omega_b: 0.49529173739994886
Error Relativo Omega_DE: 0.11970016713507274
Error Relativo ns: 0.012115965076198892
Error Relativo sigma8: 0.015575782416758334
 
COMPARACION
Comparison for Omega_m: 12.273248920276403
Comparison for h: 93.68483382532345
Comparison for Omega_b: 5.381220723393376
Comparison for Omega_DE: 242.00047752877921
Comparison for ns: 65.38295692514602
Comparison for sigma8: 3.8385494450555724


# $w_0$ and $w_a$ flat

Parameters:

1. $\Omega_{m,0}$

2. $h$

3. $\Omega_{b,0}$

4. $w_0$

5. $w_a$

6. $n_s$

7. $\sigma_8$

In [117]:
params = {'num_params': 7, 'type': 'non_standard', 'model' : 'non_ACDM_flat'}

C = Fisher(params)

In [118]:
F_3 = C.trace(Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma)

INFO:root:Calculated pair: (0, 0)
INFO:root:Calculated pair: (0, 1)
INFO:root:Calculated pair: (0, 2)
INFO:root:Calculated pair: (0, 3)
INFO:root:Calculated pair: (0, 4)
INFO:root:Calculated pair: (0, 5)
INFO:root:Calculated pair: (0, 6)
INFO:root:Calculated pair: (0, 7)
INFO:root:Calculated pair: (0, 8)
INFO:root:Calculated pair: (0, 9)
INFO:root:Calculated pair: (1, 1)
INFO:root:Calculated pair: (1, 2)
INFO:root:Calculated pair: (1, 3)
INFO:root:Calculated pair: (1, 4)
INFO:root:Calculated pair: (1, 5)
INFO:root:Calculated pair: (1, 6)
INFO:root:Calculated pair: (1, 7)
INFO:root:Calculated pair: (1, 8)
INFO:root:Calculated pair: (1, 9)
INFO:root:Calculated pair: (2, 2)
INFO:root:Calculated pair: (2, 3)
INFO:root:Calculated pair: (2, 4)
INFO:root:Calculated pair: (2, 5)
INFO:root:Calculated pair: (2, 6)
INFO:root:Calculated pair: (2, 7)
INFO:root:Calculated pair: (2, 8)
INFO:root:Calculated pair: (2, 9)
INFO:root:Calculated pair: (3, 3)
INFO:root:Calculated pair: (3, 4)
INFO:root:Calc

In [120]:
Cov = np.linalg.inv(F_3)

print('ERRORS')

print('Error Omega_m: ' + str(np.sqrt(Cov[0, 0])))
print('Error h: ' + str(np.sqrt(Cov[1, 1])))
print('Error Omega_b: ' + str(np.sqrt(Cov[2, 2])))
print('Error w0: ' + str(np.sqrt(Cov[3, 3])))
print('Error wa: ' + str(np.sqrt(Cov[4, 4])))
print('Error ns: ' + str(np.sqrt(Cov[5, 5])))
print('Error sigma8: ' + str(np.sqrt(Cov[6, 6])))

print(' ')
print('RELATIVE ERRORS')

err_Om_m = np.sqrt(Cov[0, 0]) / Omega_m0
err_h = np.sqrt(Cov[1, 1]) / h
err_Om_b = np.sqrt(Cov[2, 2]) / Omega_b0
err_w0 = np.sqrt(Cov[3, 3])
err_wa = np.sqrt(Cov[4, 4])
err_ns = np.sqrt(Cov[5, 5]) / ns
err_sig = np.sqrt(Cov[6, 6]) / sigma8

print('Error Relativo Omega_m: ' + str(err_Om_m))
print('Error Relativo h: ' + str(err_h))
print('Error Relativo Omega_b: ' + str(err_Om_b))
print('Error Relativo w0: ' + str(err_w0))
print('Error Relativo wa: ' + str(err_wa))
print('Error Relativo ns: ' + str(err_ns))
print('Error Relativo sigma8: ' + str(err_sig))

print(' ')
print('COMPARISON')

## Comparison with the paper (the numbers must be under 10)

S_Om_m = 0.044
S_h = 0.21
S_Om_b = 0.47
S_w0 = 0.16
S_wa = 0.59
S_ns = 0.038
S_sig = 0.019

print('Comparison for Omega_m: ' + str(comparison(err_Om_m, S_Om_m)))
print('Comparison for h: ' + str(comparison(err_h, S_h)))
print('Comparison for Omega_b: ' + str(comparison(err_Om_b, S_Om_b)))
print('Comparison for w0 (considering w0 = 1): ' + str(comparison(err_w0, 1)))
print('Comparison for wa (dont considering the function): ' + str(err_wa))
print('Comparison for ns: ' + str(comparison(err_ns, S_ns)))
print('Comparison for sigma8: ' + str(comparison(err_sig, S_sig)))

ERRORS
Error Omega_m: 0.03695357354791129
Error h: 0.03917409509061598
Error Omega_b: 0.058656952779373155
Error w0: 0.3862231794655483
Error wa: 1.4839409724379495
Error ns: 0.01184571436982343
Error sigma8: 0.012968864061126033
 
RELATIVE ERRORS
Error Relativo Omega_m: 0.11547991595332548
Error Relativo h: 0.058468798642710415
Error Relativo Omega_b: 1.173139055587463
Error Relativo w0: 0.3862231794655483
Error Relativo wa: 1.4839409724379495
Error Relativo ns: 0.012339285801899408
Error Relativo sigma8: 0.015893215761183865
 
COMPARISON
Comparison for Omega_m: 162.4543544393761
Comparison for h: 72.15771493204267
Comparison for Omega_b: 149.6040543803113
Comparison for w0 (considering w0 = 1): 61.37768205344517
Comparison for wa (dont considering the function): 1.4839409724379495
Comparison for ns: 67.52819525815946
Comparison for sigma8: 16.351495993769127


In [ ]:
# Considering standard

#ERRORS
#Error Omega_m: 0.04279705254934824
#Error h: 0.04562814752666197
#Error Omega_b: 0.07368988877949229
#Error w0: 0.41696014567524153
#Error wa: 1.6525779970081127
#Error ns: 0.011673369357894082
#Error sigma8: 0.012734288416901947
# 
#RELATIVE ERRORS
#Error Relativo Omega_m: 0.1337407876139799
#Error Relativo h: 0.06810171272636115
#Error Relativo Omega_b: 1.4737977755898457
#Error Relativo w0: 0.41696014567524153
#Error Relativo wa: 1.6525779970081127
#Error Relativo ns: 0.012159759747806335
#Error Relativo sigma8: 0.015605745608948465
# 
#COMPARISON
#Comparison for Omega_m: 203.95633548631795
#Comparison for h: 67.57061298744706
#Comparison for Omega_b: 213.57399480635016
#Comparison for w0 (considering w0 = 1): 58.30398543247585
#Comparison for wa (dont considering the function): 1.6525779970081127
#Comparison for ns: 68.0006322426149
#Comparison for sigma8: 17.86449679500808

# $w_0$ and $w_a$ non-flat

Parameters:

1. $\Omega_{m,0}$

2. $h$

3. $\Omega_{b,0}$

4. $\Omega_{DE,0}$

5. $w_0$

6. $w_a$

7. $n_s$

8. $\sigma_8$

In [121]:
params = {'num_params' : 8, 'type': 'standard', 'model' : 'non_ACDM_non_flat'}

D = Fisher(params)

In [122]:
F_4 = D.trace(Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma)

INFO:root:Calculated pair: (0, 0)
INFO:root:Calculated pair: (0, 1)
INFO:root:Calculated pair: (0, 2)
INFO:root:Calculated pair: (0, 3)
INFO:root:Calculated pair: (0, 4)
INFO:root:Calculated pair: (0, 5)
INFO:root:Calculated pair: (0, 6)
INFO:root:Calculated pair: (0, 7)
INFO:root:Calculated pair: (0, 8)
INFO:root:Calculated pair: (0, 9)
INFO:root:Calculated pair: (1, 1)
INFO:root:Calculated pair: (1, 2)
INFO:root:Calculated pair: (1, 3)
INFO:root:Calculated pair: (1, 4)
INFO:root:Calculated pair: (1, 5)
INFO:root:Calculated pair: (1, 6)
INFO:root:Calculated pair: (1, 7)
INFO:root:Calculated pair: (1, 8)
INFO:root:Calculated pair: (1, 9)
INFO:root:Calculated pair: (2, 2)
INFO:root:Calculated pair: (2, 3)
INFO:root:Calculated pair: (2, 4)
INFO:root:Calculated pair: (2, 5)
INFO:root:Calculated pair: (2, 6)
INFO:root:Calculated pair: (2, 7)
INFO:root:Calculated pair: (2, 8)
INFO:root:Calculated pair: (2, 9)
INFO:root:Calculated pair: (3, 3)
INFO:root:Calculated pair: (3, 4)
INFO:root:Calc

In [124]:
Cov = np.linalg.inv(F_4)

print('ERRORS')

print('Error Omega_m: ' + str(np.sqrt(Cov[0, 0])))
print('Error h: ' + str(np.sqrt(Cov[1, 1])))
print('Error Omega_b: ' + str(np.sqrt(Cov[2, 2])))
print('Error Omega_DE: ' + str(np.sqrt(Cov[3, 3])))
print('Error w0: ' + str(np.sqrt(Cov[4, 4])))
print('Error wa: ' + str(np.sqrt(Cov[5, 5])))
print('Error ns: ' + str(np.sqrt(Cov[6, 6])))
print('Error sigma8: ' + str(np.sqrt(Cov[7, 7])))

print(' ')
print('RELATIVE ERRORS')

err_Om_m = np.sqrt(Cov[0, 0]) / Omega_m0
err_h = np.sqrt(Cov[1, 1]) / h
err_Om_b = np.sqrt(Cov[2, 2]) / Omega_b0
err_Om_DE = np.sqrt(Cov[3, 3]) / Omega_DE0
err_w0 = np.sqrt(Cov[4, 4])
err_wa = np.sqrt(Cov[5, 5])
err_ns = np.sqrt(Cov[6, 6]) / ns
err_sig = np.sqrt(Cov[7, 7]) / sigma8

print('Error Relativo Omega_m: ' + str(err_Om_m))
print('Error Relativo h: ' + str(err_h))
print('Error Relativo Omega_b: ' + str(err_Om_b))
print('Error Relativo Omega_DE: ' + str(err_Om_DE))
print('Error Relativo w0: ' + str(err_w0))
print('Error Relativo wa: ' + str(err_wa))
print('Error Relativo ns: ' + str(err_ns))
print('Error Relativo sigma8: ' + str(err_sig))

print(' ')
print('COMPARISON')

## Comparison with the paper (the numbers must be under 10)

S_Om_m = 0.047
S_h = 0.21
S_Om_b = 0.48
S_Om_DE = 0.2
S_w0 = 0.23
S_wa = 1.85
S_ns = 0.038
S_sig = 0.02

print('Comparison for Omega_m: ' + str(comparison(err_Om_m, S_Om_m)))
print('Comparison for h: ' + str(comparison(err_h, S_h)))
print('Comparison for Omega_b: ' + str(comparison(err_Om_b, S_Om_b)))
print('Comparison for w0 (considering w0 = 1): ' + str(comparison(err_w0, 1)))
print('Comparison for wa (dont considering the function): ' + str(err_wa))
print('Comparison for ns: ' + str(comparison(err_ns, S_ns)))
print('Comparison for sigma8: ' + str(comparison(err_sig, S_sig)))

ERRORS
Error Omega_m: 0.045961727764737476
Error h: 0.048622338411176656
Error Omega_b: 0.28112399800262305
Error Omega_DE: 2.810794748784581
Error w0: 1.286958901871165
Error wa: 9.582094086972864
Error ns: 0.011673994962439049
Error sigma8: 0.012738008168114082
 
RELATIVE ERRORS
Error Relativo Omega_m: 0.14363039754355536
Error Relativo h: 0.07257065434503979
Error Relativo Omega_b: 5.622479960052461
Error Relativo Omega_DE: 4.133521712699971
Error Relativo w0: 1.286958901871165
Error Relativo wa: 9.582094086972864
Error Relativo ns: 0.012160411419207344
Error Relativo sigma8: 0.015610304127590787
 
COMPARISON
Comparison for Omega_m: 205.5965905182029
Comparison for h: 65.44254554998106
Comparison for Omega_b: 1071.349991677596
Comparison for w0 (considering w0 = 1): 28.69589018711649
Comparison for wa (dont considering the function): 9.582094086972864
Comparison for ns: 67.99891731787542
Comparison for sigma8: 21.94847936204607


# $w_0$, $w_a$ and $\gamma$ non-flat

Parameters:

1. $\Omega_{m,0}$

2. $h$

3. $\Omega_{b,0}$

4. $\Omega_{DE,0}$

5. $w_0$

6. $w_a$

7. $n_s$

8. $\sigma_8$

9. $\gamma$

In [130]:
params = {'num_params' : 9, 'type': 'non_standard', 'model' : 'non_ACDM_non_flat_gamma'}

E = Fisher(params)

In [131]:
F_5 = E.trace(Omega_m0, h, Omega_b0, Omega_DE0, w0, wa, ns, sigma8, gamma)

INFO:root:Calculated pair: (0, 0)
INFO:root:Calculated pair: (0, 1)
INFO:root:Calculated pair: (0, 2)
INFO:root:Calculated pair: (0, 3)
INFO:root:Calculated pair: (0, 4)
INFO:root:Calculated pair: (0, 5)
INFO:root:Calculated pair: (0, 6)
INFO:root:Calculated pair: (0, 7)
INFO:root:Calculated pair: (0, 8)
INFO:root:Calculated pair: (0, 9)
INFO:root:Calculated pair: (1, 1)
INFO:root:Calculated pair: (1, 2)
INFO:root:Calculated pair: (1, 3)
INFO:root:Calculated pair: (1, 4)
INFO:root:Calculated pair: (1, 5)
INFO:root:Calculated pair: (1, 6)
INFO:root:Calculated pair: (1, 7)
INFO:root:Calculated pair: (1, 8)
INFO:root:Calculated pair: (1, 9)
INFO:root:Calculated pair: (2, 2)
INFO:root:Calculated pair: (2, 3)
INFO:root:Calculated pair: (2, 4)
INFO:root:Calculated pair: (2, 5)
INFO:root:Calculated pair: (2, 6)
INFO:root:Calculated pair: (2, 7)
INFO:root:Calculated pair: (2, 8)
INFO:root:Calculated pair: (2, 9)
INFO:root:Calculated pair: (3, 3)
INFO:root:Calculated pair: (3, 4)
INFO:root:Calc

In [132]:
Cov = np.linalg.inv(F_5)

print('ERRORS')

print('Error Omega_m: ' + str(np.sqrt(Cov[0, 0])))
print('Error h: ' + str(np.sqrt(Cov[1, 1])))
print('Error Omega_b: ' + str(np.sqrt(Cov[2, 2])))
print('Error Omega_DE: ' + str(np.sqrt(Cov[3, 3])))
print('Error w0: ' + str(np.sqrt(Cov[4, 4])))
print('Error wa: ' + str(np.sqrt(Cov[5, 5])))
print('Error ns: ' + str(np.sqrt(Cov[6, 6])))
print('Error sigma8: ' + str(np.sqrt(Cov[7, 7])))
print('Error gamma: ' + str(np.sqrt(Cov[8, 8])))

print(' ')
print('RELATIVE ERRORS')

err_Om_m = np.sqrt(Cov[0, 0]) / Omega_m0
err_h = np.sqrt(Cov[1, 1]) / h
err_Om_b = np.sqrt(Cov[2, 2]) / Omega_b0
err_Om_DE = np.sqrt(Cov[3, 3]) / Omega_DE0
err_w0 = np.sqrt(Cov[4, 4])
err_wa = np.sqrt(Cov[5, 5])
err_ns = np.sqrt(Cov[6, 6]) / ns
err_sig = np.sqrt(Cov[7, 7]) / sigma8
err_gamma = np.sqrt(Cov[8, 8]) / gamma

print('Error Relativo Omega_m: ' + str(err_Om_m))
print('Error Relativo h: ' + str(err_h))
print('Error Relativo Omega_b: ' + str(err_Om_b))
print('Error Relativo Omega_DE: ' + str(err_Om_DE))
print('Error Relativo w0: ' + str(err_w0))
print('Error Relativo wa: ' + str(err_wa))
print('Error Relativo ns: ' + str(err_ns))
print('Error Relativo sigma8: ' + str(err_sig))
print('Error Relativo gamma: ' + str(err_gamma))

print(' ')
print('COMPARISON')

## Comparison with the paper (the numbers must be under 10)

S_Om_m = 0.055
S_h = 0.23
S_Om_b = 0.5
S_Om_DE = 0.57
S_w0 = 0.25
S_wa = 1.9
S_ns = 0.037
S_sig = 0.029
S_gamma = 1.1

print('Comparison for Omega_m: ' + str(comparison(err_Om_m, S_Om_m)))
print('Comparison for h: ' + str(comparison(err_h, S_h)))
print('Comparison for Omega_b: ' + str(comparison(err_Om_b, S_Om_b)))
print('Comparison for w0 (considering w0 = 1): ' + str(comparison(err_w0, 1)))
print('Comparison for wa (dont considering the function): ' + str(err_wa))
print('Comparison for ns: ' + str(comparison(err_ns, S_ns)))
print('Comparison for sigma8: ' + str(comparison(err_sig, S_sig)))
print('Comparison for sigma8: ' + str(comparison(err_gamma, S_gamma)))

ERRORS
Error Omega_m: 0.04528535745063641
Error h: 0.04725929279287451
Error Omega_b: 0.3321050504999246
Error Omega_DE: 3.2692009569827656
Error w0: 1.474366874054996
Error wa: 10.999027492514564
Error ns: 0.01262492393064241
Error sigma8: 0.013724853310554872
Error gamma: 0.4433385779023396
 
RELATIVE ERRORS
Error Relativo Omega_m: 0.14151674033731934
Error Relativo h: 0.0705362578998127
Error Relativo Omega_b: 6.642101009998492
Error Relativo Omega_DE: 4.807648493263726
Error Relativo w0: 1.474366874054996
Error Relativo wa: 10.999027492514564
Error Relativo ns: 0.013150962427752511
Error Relativo sigma8: 0.0168196731746996
Error Relativo gamma: 0.812787392820956
 
COMPARISON
Comparison for Omega_m: 157.3031642496715
Comparison for h: 69.33206178269013
Comparison for Omega_b: 1228.4202019996985
Comparison for w0 (considering w0 = 1): 47.4366874054996
Comparison for wa (dont considering the function): 10.999027492514564
Comparison for ns: 64.45685830337159
Comparison for sigma8: 42.0